In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import cv2
from PIL import Image

In [ ]:
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
df

In [ ]:
data_dir = '../input/rsna-miccai-png/train'
patients = sorted(os.listdir(data_dir))
patients[:5]

In [ ]:
data_dir = '../input/rsna-miccai-png/test'
patients_test = sorted(os.listdir(data_dir))
patients_test[:5]

In [ ]:
id = []
for i in patients:
    id.append(int(i))
id[:5]

In [ ]:
train_dataset = pd.DataFrame({'path' : glob('../input/rsna-miccai-png/train/00000/FLAIR/*.png'),
                              'label' : 1})

for i, patient in enumerate(patients[1:]):
    data_slice = pd.DataFrame({'path' : glob('../input/rsna-miccai-png/train/' + patient +'/FLAIR/*.png'),
                               'label' : df['MGMT_value'][i+1]})
    train_dataset = pd.concat([train_dataset, data_slice])
    
train_dataset = train_dataset.reset_index(drop = True)
train_dataset['label'] = train_dataset['label'].astype(str)
train_dataset

In [ ]:
test_dataset = pd.DataFrame({'path' : glob('../input/rsna-miccai-png/test/00001/FLAIR/*.png')})

for i, patient in enumerate(patients_test[1:]):
    data_slice = pd.DataFrame({'path' : glob('../input/rsna-miccai-png/test/' + patient +'/FLAIR/*.png')})
    test_dataset = pd.concat([test_dataset, data_slice])
    
test_dataset = test_dataset.reset_index(drop = True)

test_dataset

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid = train_test_split(train_dataset, stratify = train_dataset['label'], random_state = 42, test_size = 0.2)

In [ ]:
import tensorflow as tf
def get_model():
    
    base_model = tf.keras.applications.EfficientNetB7(include_top=False, weights="../input/keras-pretrained-models/EfficientNetB7_NoTop_ImageNet.h5")


    base_model.trainable = False


    inputs = tf.keras.layers.Input(shape=(150, 150, 3), name="input_layer")



    x = base_model(inputs)

    print(f"Shape after base_model: {x.shape}")


    x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
    print(f"After GlobalAveragePooling2D(): {x.shape}")


    outputs = tf.keras.layers.Dense(2, activation="softmax", name="output_layer")(x)


    model = tf.keras.Model(inputs, outputs)
    return  model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

from keras import *
from keras.layers import *
from tensorflow.keras.applications import EfficientNetB0
from keras.callbacks import *

idg = ImageDataGenerator(horizontal_flip = True)
idg2 = ImageDataGenerator()

train_dataset = idg.flow_from_dataframe(x_train, x_col = 'path', y_col = 'label', target_size=(150, 150), batch_size = 64)
valid_dataset = idg2.flow_from_dataframe(x_valid, x_col = 'path', y_col = 'label', target_size=(150, 150), batch_size = 64)

#efn = EfficientNetB0(include_top = False, pooling = 'avg', input_shape=(150, 150, 3))
es = EarlyStopping(patience = 2, restore_best_weights = True)
rl = ReduceLROnPlateau(patience = 1, factor = 0.2, verbose = 1)

model = get_model()


model.compile(metrics = ['acc'], loss = 'categorical_crossentropy', optimizer = 'adam')

model.fit(train_dataset, validation_data = valid_dataset, epochs = 2, callbacks = [es, rl])

test_generator = idg2.flow_from_dataframe(test_dataset, x_col = 'path', y_col = None, target_size = (150, 150), batch_size = 64, class_mode = None, shuffle = False)

result = model.predict(test_generator, verbose = True, workers = 2)

In [ ]:
model.save("rsna_baseline_model") 

In [ ]:
result

In [ ]:
final_result = []
i = 0
for patient in patients_test:
    length = len(glob('../input/rsna-miccai-png/test/' + patient +'/FLAIR/*.png'))
    final_result.append(result[i:i+length, 1].sum()/length)
    i += length
    
final_result